In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
from werkzeug.utils import secure_filename
from flask_bcrypt import bcrypt
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

app = Flask(__name__)
app.secret_key = 'supersecretmre'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    name = db.Column(db.String(120), nullable=False)

    def __repr__(self):
        return f'<User {self.username}>'
    
    def __init__(self, username, password, email, name):
        self.username = username
        self.password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
        self.email = email
        self.name = name

    def check_password(self, password):
        return bcrypt.checkpw(password.encode('utf-8'), self.password.encode('utf-8'))
    

with app.app_context():
    db.create_all()


df = pd.read_csv('jobs.csv')

@app.route('/')
def index():
    # flash('Welcome to the Flask App', 'info')
    return render_template('index.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first() or User.query.filter_by(email=username).first()
        if user and user.check_password(password):
            flash('Login successful', 'success')
            return redirect(url_for('index'))
        else:
            flash('Invalid credentials', 'danger')
            return redirect(url_for('login'))
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        email = request.form['email']
        name = request.form['name']

        # Check if user already exists
        existing_user = User.query.filter_by(username=username).first() or User.query.filter_by(email=email).first()
        if existing_user:
            flash('Username already exists', 'danger')
            return redirect(url_for('register'))

        # Create new user
        new_user = User(username=username, password=password, email=email, name=name)
        db.session.add(new_user)
        db.session.commit()
        flash('Registration successful', 'success')
        return redirect(url_for('login'))
    return render_template('Registration.html')



#Graphs functions

#common job titles
def common_job_titles():
    fig1 = go.Figure()
    top_titles = df['title'].value_counts().nlargest(15).reset_index()
    top_titles.columns = ['Job Title', 'Count']

    fig1 = px.bar(top_titles, x='Job Title', y='Count',
              title="Top 15 Most Common Job Titles",
              color='Count', text='Count')
    fig1.update_layout(xaxis_tickangle=-45)

    graph1_html = pio.to_html(fig1, full_html=False)
    return graph1_html

# Distribution of Employment Types
def employment_type_distribution():
    fig2 = go.Figure()
    employment_dist = df['Employment type'].value_counts().reset_index()
    employment_dist.columns = ['Employment Type', 'Count']

    fig2 = px.pie(employment_dist, names='Employment Type', values='Count',
              title="Distribution of Employment Types",
              hole=0.3)
    graph2_html = pio.to_html(fig2, full_html=False)
    return graph2_html   # Donut style


# Distribution of Seniority Levels
def seniority_level_distribution():
    fig3 = go.Figure()
    fig3 = px.histogram(df, x='Seniority level',
                    title="Distribution of Seniority Levels",
                    color='Seniority level')
    fig3.update_layout(xaxis_title='Seniority Level', yaxis_title='Count')
    graph3_html = pio.to_html(fig3, full_html=False)
    return graph3_html   # Donut style

   


# Seniority Level by Employment Type
def seniority_level_by_employment_type():
    fig4 = go.Figure()
    combo = df.groupby(['Employment type', 'Seniority level']).size().reset_index(name='Count')

    fig4 = px.bar(combo, x='Employment type', y='Count', color='Seniority level',
              title="Seniority Level by Employment Type",
              barmode='stack')
    graph4_html = pio.to_html(fig4, full_html=False)
    return graph4_html   # Donut style



# Top 10 Companies by Job Postings
def top_companies():
    fig5 = go.Figure()  
    top_companies = df['company'].value_counts().nlargest(10).reset_index()
    top_companies.columns = ['Company', 'Postings']

    fig5 = px.pie(top_companies, names='Company', values='Postings',
                title="Top 10 Companies by Job Postings",
                hole=0.4)
    graph5_html = pio.to_html(fig5, full_html=False)
    return graph5_html   # Donut style

#industry analysis page routes
def industries_by_job_volume():
    fig1 = go.Figure()  
    top_industries = df['Industries'].value_counts().nlargest(15).reset_index()
    top_industries.columns = ['Industry', 'Job Count']

    fig1 = px.bar(top_industries, x='Industry', y='Job Count',
            title="Top 15 Industries by Job Postings",
            color='Job Count', text='Job Count')
    fig1.update_layout(xaxis_tickangle=-45)
    graph1_html = pio.to_html(fig1, full_html=False)
    return graph1_html   # Donut style




# Treemap of Job Functions Within Industries
def job_functions_within_industries():
    fig2 = go.Figure()
    treemap_df = df.groupby(['Industries', 'Job function']).size().reset_index(name='Count')

    fig2 = px.treemap(treemap_df, path=['Industries', 'Job function'], values='Count',
        title="Treemap of Job Functions Within Industries")
    graph2_html = pio.to_html(fig2, full_html=False) 
    return graph2_html   # Donut style




# Heatmap – Cross-tab of job function vs. industry
def job_function_vs_industry_heatmap():
    fig3 = go.Figure()
    heatmap_data = df.groupby(['Job function', 'Industries']).size().reset_index(name='Count')
    heatmap_pivot = heatmap_data.pivot(index='Job function', columns='Industries', values='Count').fillna(0)

    fig3 = px.imshow(heatmap_pivot,
                 labels=dict(x="Industry", y="Job Function", color="Job Count"),
                 title="Heatmap – Job Function by Industry")
    fig3.update_xaxes(tickangle=-45)
    graph3_html = pio.to_html(fig3, full_html=False)
    return graph3_html   # Donut style



# Bubble Chart – Job function popularity (size = number of postings)
def job_function_popularity():
    fig4 = go.Figure()
    job_function_count = df['Job function'].value_counts().reset_index()
    job_function_count.columns = ['Job Function', 'Count']

    fig4 = px.scatter(job_function_count, x='Job Function', y='Count',
                  size='Count', color='Job Function',
                  title="Job Function Popularity",
                  size_max=60)
    fig4.update_layout(xaxis_tickangle=-45)
    graph4_html = pio.to_html(fig4, full_html=False)
    return graph4_html   # Donut style



#Pie Chart – Share of top 10 job functions
def job_function_share():
    fig5 = go.Figure()
    top_functions = df['Job function'].value_counts().nlargest(10).reset_index()
    top_functions.columns = ['Job Function', 'Count']

    fig5 = px.pie(top_functions, names='Job Function', values='Count',
              title="Top 10 Job Functions Distribution")
    graph5_html = pio.to_html(fig5, full_html=False)
    return graph5_html   # Donut style



                                                                       # Salary_Distribution_&_Trends file
# Box Plot – Salary Ranges by Job Function
def salary_ranges_by_job_function():
    fig1 = go.Figure()
    df['sal_low'] = pd.to_numeric(df['sal_low'], errors='coerce')
    df['sal_high'] = pd.to_numeric(df['sal_high'], errors='coerce')
    df['avg_salary'] = (df['sal_low'] + df['sal_high']) / 2
    fig1 = px.box(df, x='Job function', y='avg_salary', points='all',
              title='Salary Ranges by Job Function',
              color='Job function')
    fig1.update_layout(xaxis_tickangle=-45, yaxis_title='Average Salary')
    graph1_html = pio.to_html(fig1, full_html=False)
    return graph1_html   # Donut style




# Violin Plot – Salary Spread by Seniority Level
def salary_spread_by_seniority_level():
    fig2 = go.Figure()
    fig2 = px.violin(df, x='Seniority level', y='avg_salary', box=True, points="all",
                 title='Salary Distribution by Seniority Level',
                 color='Seniority level')
    fig2.update_layout(xaxis_tickangle=-45, yaxis_title='Average Salary')
    
    graph2_html = pio.to_html(fig2, full_html=False)
    return graph2_html   # Donut style




# Histogram – Distribution of Average Salaries
def salary():
    fig3 = go.Figure()
    fig3 = px.histogram(df, x='avg_salary',
                    nbins=30,
                    title='Distribution of Average Salaries',
                    color_discrete_sequence=['indianred'])
    fig3.update_layout(xaxis_title='Average Salary', yaxis_title='Job Count')
    
    graph3_html = pio.to_html(fig3, full_html=False)
    return graph3_html   # Donut style



# Scatter Plot – Experience vs. Average Salary
def experience_vs_salary():
    fig4 = go.Figure()
    df['months_experience'] = pd.to_numeric(df['months_experience'], errors='coerce')

    fig4 = px.scatter(df, x='months_experience', y='avg_salary',
                  title='Experience vs. Average Salary',
                  trendline='ols',  # adds regression line
                  color='Job function')
    fig4.update_layout(xaxis_title='Months of Experience', yaxis_title='Average Salary')
    graph4_html = pio.to_html(fig4, full_html=False)
    return graph4_html   # Donut style





# Bar Chart – Top 15 Highest Paying Job Titles
def top_15_highest_paying_job_titles():
    fig5 = go.Figure()
    title_salary = df.groupby('title')['avg_salary'].mean().nlargest(15).reset_index()

    fig5 = px.bar(title_salary, x='title', y='avg_salary',
              title='Top 15 Highest Paying Job Titles',
              color='avg_salary', text='avg_salary')
    fig5.update_layout(xaxis_tickangle=-45, yaxis_title='Average Salary')
    
    graph5_html = pio.to_html(fig5, full_html=False)
    return graph5_html   # Donut style




#analysis page routes 
@app.route('/job_analysis')
def job_analysis():
    # Generate the graphs
    graph1_html = common_job_titles()
    graph2_html = employment_type_distribution()
    graph3_html = seniority_level_distribution()
    graph4_html = seniority_level_by_employment_type()
    graph5_html = top_companies()

    # Add more graphs as needed
    return render_template('job_analysis.html', graph1_html=graph1_html, graph2_html=graph2_html, graph3_html=graph3_html, graph4_html=graph4_html, graph5_html=graph5_html)



#analysis page routes
@app.route('/industry_function_insights')
def industries_by_volume():
    # Generate the graphs
    graph1_html = industries_by_job_volume()
    graph2_html = job_functions_within_industries()
    graph3_html = job_function_vs_industry_heatmap()
    graph4_html = job_function_popularity()
    graph5_html = job_function_share()
    # Add more graphs as needed
    return render_template('industry_function_insights.html', graph1_html=graph1_html,graph2_html=graph2_html, graph3_html=graph3_html, graph4_html=graph4_html, graph5_html=graph5_html)



@app.route('/salary_distribution')
def salary_distribution():
    # Generate the graphs
    graph1_html = salary_ranges_by_job_function()
    graph2_html = salary_spread_by_seniority_level()
    graph3_html = salary()
    graph4_html = experience_vs_salary()
    graph5_html = top_15_highest_paying_job_titles()

    # Add more graphs as needed
    return render_template('salary_distribution.html', graph1_html=graph1_html, graph2_html=graph2_html, graph3_html=graph3_html, graph4_html=graph4_html, graph5_html=graph5_html) 

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.60.9:5000
Press CTRL+C to quit
